In [60]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn import neighbors
from sklearn.metrics import confusion_matrix, classification_report

In [61]:
import json
with open("the_final_data.json", 'r', encoding='UTF-8') as f:
    data = json.load(f)

In [62]:
NEW_COL = ['name', 'code', 'time', 'price', 'time_1', 'price_1', 'price_dif_1',
       'sell_1', 'buy_1', 'volume_1', 'variation_1', 'post_num_1',
       'unique_id_1', 'click_1', 'like_1', 'dislike_1', 'time_2', 'price_2',
       'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 'variation_2',
       'post_num_2', 'unique_id_2', 'click_2', 'like_2', 'dislike_2', 'time_3',
       'price_3', 'price_dif_3', 'sell_3', 'buy_3', 'volume_3', 'variation_3',
       'post_num_3', 'unique_id_3', 'click_3', 'like_3', 'dislike_3',
       'mkt_cap', 'kospi', 'kosdaq', 'trash', 'yesterday_closing_price',
       'is_maximum', 'is_minimum', 'price_volatility', 'price_trend',
       'average_price_volatility', 'sell_minus_buy_1', 'sell_minus_buy_2',
       'sell_minus_buy_3', 'is_price_gap_stable', 'price_gap_volatility',
       'is_like_higher', 'volume_trend', 'post_num_trend', 'unique_id_trend',
       'click_trend', 'price_increase', 'did_price_increase', 'did_price_033',
       'did_price_100', 'did_price_150', 'kospi_ind', 'kosdaq_ind',
       'time_slot', 'ko_inter', 'early_mor', 'morning', 'lunch', 'afternoon',
       'late', 'mkt_change', 'alpha', 'per_now', 'did_opening_price_increase']
'''
OLD_COL = ['name', 'code', 'time', 'price', 'time_1', 'price_1', 'price_dif_1',
       'sell_1', 'buy_1', 'volume_1', 'variation_1', 'post_num_1',
       'unique_id_1', 'click_1', 'like_1', 'dislike_1', 'time_2', 'price_2',
       'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 'variation_2',
       'post_num_2', 'unique_id_2', 'click_2', 'like_2', 'dislike_2', 'time_3',
       'price_3', 'price_dif_3', 'sell_3', 'buy_3', 'volume_3', 'variation_3',
       'post_num_3', 'unique_id_3', 'click_3', 'like_3', 'dislike_3',]
'''

df = pd.DataFrame(data, columns = NEW_COL)
df = df.dropna(axis=0, how='any')

In [63]:
df = df[df.did_opening_price_increase == 1]

In [64]:
time_filter = (df['time'].str.startswith("2018-02-21")) | \
              (df['time'].str.startswith("2018-02-22")) | \
              (df['time'].str.startswith("2018-02-23")) | \
              (df['time'].str.startswith("2018-02-26")) 
        
train_df = df[time_filter].reset_index(drop = True)
test_df = df[~time_filter].reset_index(drop = True)

In [65]:
COL_DROP = ['name', 'code', 'time', 'price', 'time_1', 'time_2', 'time_3',
            'did_price_increase', 'did_price_033', 'did_price_100', 'did_price_150', 
            'price_increase', 'did_opening_price_increase']
X = df.drop(COL_DROP, axis = 1)
y_inc = df['price_increase']
y_0 = df['did_price_increase']
y_33 = df['did_price_033']
y_100 = df['did_price_100']
y_150 = df['did_price_150']
X_train = train_df.drop(COL_DROP, axis = 1)
X_test = test_df.drop(COL_DROP, axis = 1)
y_train_in = train_df['price_increase']
y_train_0 = train_df['did_price_increase']
y_train_33 = train_df['did_price_033']
y_train_100 = train_df['did_price_100']
y_train_150 = train_df['did_price_150']
y_test_in = test_df['price_increase']
y_test_0 = test_df['did_price_increase']
y_test_33 = test_df['did_price_033']
y_test_100 = test_df['did_price_100']
y_test_150 = test_df['did_price_150']

In [49]:
knn = neighbors.KNeighborsClassifier(n_neighbors = 52)
y_pred_0 = knn.fit(X_train, y_train_0).predict(X_test)

In [50]:
result_matrix = confusion_matrix(y_test_0, y_pred_0)
result_matrix

array([[1515,   12],
       [ 873,   14]], dtype=int64)

In [51]:
print(classification_report(y_test_0, y_pred_0))

             precision    recall  f1-score   support

          0       0.63      0.99      0.77      1527
          1       0.54      0.02      0.03       887

avg / total       0.60      0.63      0.50      2414



In [52]:
y_port = y_test_in[y_pred_0 == 1]
y_port.mean()

0.18136878792692307

In [53]:
y_port.mean()

0.18136878792692307

In [54]:
y_inc.mean()

-0.028556283185194015

In [66]:
def get_neighbour_mean(num):
    
    knn = neighbors.KNeighborsClassifier(n_neighbors = num)
    y_pred = knn.fit(X_train, y_train_0).predict(X_test)
    y_port = y_test_in[y_pred == 1]
    rv = y_port.mean()
    
    return rv    

In [67]:
def get_best_neighbour(num):
    rv_max = 0
    max_id = 1
    for i in range(1, num):
        rv_max = max(get_neighbour_mean(i), rv_max)
        if rv_max == get_neighbour_mean(i):
            max_id = i
    return max_id, rv_max

In [68]:
get_best_neighbour(100)

(1, 0)